In [27]:
import os
import pickle
import math

from dotenv import load_dotenv
from domain.vehicle import Vehicle
from abi.libs.rust_solver_lib import GAParameters
from abi.libs.rust_solver_lib import GraspParameters
from abi.libs.rust_solver_lib import RustSolverLib
from data.stops import stops
from distancesAPI import MapsAPI

%reload_ext autoreload
%autoreload 2

load_dotenv(override=True)

True

In [28]:
refetch_distances = False
calculate_time = False
plot_markers = False

In [29]:
addresses = [x.address for x in stops]
mapsAPI = MapsAPI(os.getenv('API_KEY'))
address_chunks = MapsAPI.split_in_chunks(addresses, 25)
distances = mapsAPI.split_distance_request(address_chunks, calculateTime = calculate_time) if refetch_distances else MapsAPI.get_from_cache()
distances = MapsAPI.convert_to_symmetric(distances)

In [30]:
with open('cache/distance_matrix', 'rb') as f:
    best_ever = pickle.load(f)

In [31]:
def insert_ids_in_distances(distances):
    return [[src_id, dest_id, distance] for src_id, sublist in enumerate(distances) for dest_id, distance in enumerate(sublist)]


In [87]:
number_of_routes = 2
distances_with_ids = insert_ids_in_distances(distances)
vehicles = [Vehicle(0, 1000), Vehicle(1, 1000), Vehicle(2, 1000), Vehicle(3, 1000)]

ga_parameters = GAParameters(
    population_size=200,
    elite_size=8,
    mutation_rate=0.05,
    local_search_rate=0.2,
    max_crossover_tries=10,
    max_generations=1000,
)

grasp_parameters = GraspParameters(
    rcl_size=math.floor(0.3 * len(stops)), max_improvement_times=10
)

relative_lib_path = f"../{os.getenv('RUST_SOLVER_LIB')}"
rust_lib = RustSolverLib(os.path.abspath(relative_lib_path))

rust_result = rust_lib.run_grasp_genetic_solver(
    vehicles[:number_of_routes], stops, distances_with_ids, grasp_parameters, ga_parameters
)

rust_addresses = [[stop.address for stop in route.stops] for route in rust_result]

print(sum(x.total_distance for x in rust_result))
print(*[[stop.id for stop in route.stops] for route in rust_result], sep="\n")

32415.0
[0, 13, 18, 19, 25, 15, 16, 12, 14, 3, 4, 6, 5, 10, 7, 8, 9, 31, 0]
[0, 1, 2, 28, 29, 30, 32, 26, 24, 22, 23, 17, 21, 20, 27, 11, 0]


In [42]:
best_n2 = [
    [0, 14, 3, 4, 6, 5, 7, 10, 20, 21, 16, 31, 28, 29, 30, 32, 0],
    [0, 13, 15, 25, 26, 24, 22, 23, 18, 17, 19, 27, 9, 8, 12, 11, 1, 2, 0],
]

In [43]:
best_n3 = [
    [0, 29, 31, 19, 28, 15, 23, 30, 24, 22, 18],
    [0, 3, 16, 21, 14, 13, 10, 27, 25, 6, 7, 9, 17, 8, 20, 2, 11, 1, 12],
    [0, 5, 26, 4, 32],
]

In [77]:
best_n4 = [
    [0, 7, 5, 8, 9, 10, 27, 18, 24, 16, 20, 0],
    [0, 11, 12, 1, 21, 23, 26, 0],
    [0, 6, 4, 3, 14, 17, 19, 22, 25, 0],
    [0, 2, 31, 28, 29, 30, 32, 15, 13, 0],
]

In [83]:
def recover_addresses(solution: list[list[int]]):
    return [[stops[stop_id].address for stop_id in route] for route in solution]

In [45]:
# with open('cache/best_ever', 'wb'):
#     pickle.dumps(best)

In [88]:
route_addresses = rust_addresses
imageCount = 1
for route in route_addresses:
    directions = mapsAPI.directions(route[0], route[0], route[1: 25])
    mapsAPI.save_to_image(directions, f"n{number_of_routes}_route_{imageCount}")
    imageCount += 1

In [89]:
if plot_markers:
    plot = mapsAPI.plot_markers(addresses)
    with open('test.jpg', 'wb') as f:
        for chunk in plot:
            f.write(chunk)